In [ ]:
"""server ne reuest suni lekin accept nhi ki and block kar diya scraping karne se Bahut websites scraping block karti hain — jaise PakWheels, OLX, LinkedIn, etc.
Agar aap requests.get() se hit karte ho, wo block kar dete hain → 403."""

# jab ham requests.get(url) se hit karte hain yani request bhejte hain tu kuch website allow kardeti hain and response 200 ajata hai mtlb sab sahi hua, lekin kuch website allow nhi karti and block kardeti hain phir 403 forbidden show karti hain kyun ke us ko lagta hai ham browser ki tarha request nhi bhej rahey, Server ko lagta hai ke request bot ne bheji hai isi liyen block kar deta hai tu ham ek headers banate hain and us mein cheexein likhte hain phir jab requests.get(url,headers=headers) karte hain tu response 200 ajata hai kyun ke Isse website ko lagta hai ke browser request bhej raha, script nahi.
all_cars=[]
for page in range(1,11):
    url = f"https://www.pakwheels.com/used-cars/search/-/ct_karachi/?page={page}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }
    response = requests.get(url,headers=headers)  
    # print(response) # response 200 mtlb reuest accept karli ab server hamein do cheezein bhejta hai pheli: Response Headers and dusri: Response Body 
    # for key, value in response.headers.items(): # yeh karne se pata chal ke content type kiya hai, mtlb server kis type ka data bhej raha hai lekin server jo hai confuse karne ke liyen bhi galat type batat hai yani bolega javascript bhej html raha hai isi liyen ham us data ko 100 percent confirm karne ke liyen us ke andar ka content check karen gey 
    #     if key == 'Content-Type':
    #         print(f'{key}: {value}')
    # print(response.text[:500])  pata chal server fool bana raha tha bhej wo html mein raha hai tu isis liyen ham .text kar ke get karengey 

    # response.text:  but .text us data ko string mein conert kar deta hai and raw data deta hai, Matlab server se jo .text mila, wo bas ek raw string hai — HTML, JSON, ya JavaScript. Usme se specific information nikalne ke liye parse karte hain.

    soup = BeautifulSoup(response.text, 'html.parser')
    all_uls = soup.find_all('ul',class_='list-unstyled search-results search-results-mid next-prev car-search-results') # saare ul lake dega jis mein class attribute ki value exactly yeh (list-unstyled search-results search-results-mid next-prev car-search-results) hogi
    
    for ul in all_uls:
        all_lis = ul.select('li.classified-listing') # yeh saare lis lake dega jo us particular ul mein hain and un ki class ke naam mein classified-listing ata ho khain bhi 

        for li in all_lis:
            script_tag = li.find('script')
            if script_tag:
                # print(script_tag.text) # .text jo hai wo string wrapper mein daal deta hai jis wajha se data type string ban jati hai 
                data = json.loads(script_tag.text)
                new_dict = {
                    'name':data.get('description'),
                    'year':data.get('modelDate'),
                    'brand':data.get('manufacturer'),
                    'mileage':data.get('mileageFromOdometer'),
                    'prices':data.get('offers',{}).get('price'),
                    'engine':data.get('vehicleEngine',{}).get('engineDisplacement','None'),
                    'vehicle_Transmission':data.get('vehicleTransmission'),
                    'fuel_type':data.get('fuelType')
                }
                
                all_cars.append(json.dumps(new_dict))
        
  
pd.DataFrame({'cars_data':all_cars}).to_csv('./output/raw.csv',index=False)

cars = []
for x in all_cars:
    cars.append(json.loads(x))

df = pd.DataFrame(cars)

df



# json ham file se bhi read kar sakte and string ke andar mojod json ko bhi read kar sakte hain, file extension kuch bhi ho agar us mein valid json hai tu ham data = json.load(script_tag) karengey tu data variable mein ek dict ya list ajayega, agar string mein wrap json ho tu us ke liyen data = json.loads(script_tag) karengey tab bhi data variable mein dict ya list hi ayega 





 

In [ ]:
# JAB RAW DATA MIL GAYA AND US RAW DATA KO CSV MEIN STORE BHI KARA DIYA AB TRANSFORM KA PART ATA HAI JSI MEIN SAB SE PHELA KAAM RAW DATA KO DATAFRAME BANANA HAI US KE BAD DATA KO UNDERSTAND KARNA HAI 

# basic info

# print(df.shape,end='\n')
# print(df.head(),end='\n')
# print(df.dtypes,end='\n')
# print(df.columns,end='\n')

# ab dekhengey ke kitne null hain har column mein  yad rahey yeh sirf  (np.nan = NaN, None, pd.NA = <NA>, float("nan") = NaN) yehi detect karega

null_string = ["", " ", "N/A", "n/a", "NA", "None", "none", "null", "Null", "unknown", "-", "--"]

df = df.replace(null_string, np.nan)


# df.isnull().sum() # yeh har column mein jitne missing value hain wo batayega 

df.isnull().sum().sum() # yeh poore dataframe mein jitni missing value hai us ka total dega

# print(df['mileage'].sample(10)) # smaple random 10 row lekar ayega 
# print(df['engine'].sample(10))
# print(df['prices'].sample(10))


# NORMALIZE COLUMN NAME MTLB SAB KE NAAM LOWER CASE MEIN AND DO WORD MEIN GAP KI BAJAYE UNDERSCORE

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
# now use: description, year, brand, mileage, prices, engine, vehicle_transmission, fuel_type
df.columns


# HANDLING MISSING VALUES 
df['brand'] = df['brand'].fillna('unknown')

df.isnull().sum().sum()



# CLEAN AND CONVERT NUMERIC COLUMNS 

df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64') # pd.to_numeric se agar missing value hogi jis row mein us ko kuch nhi karega and jahan value hogi wahan int kar dega
df['mileage'] = df['mileage'].str.replace('km','').str.replace(',','').str.strip()
df['mileage'] = pd.to_numeric(df['mileage'],errors='coerce').astype('Int64')
df['prices'] = pd.to_numeric(df['prices'],errors='coerce').astype('Int64')
df['engine'] = df['engine'].str.replace('cc','').str.strip()
df['engine'] = pd.to_numeric(df['engine'],errors='coerce').astype('Int64')
df['name'] = df['name'].str.replace('for sale in Karachi','')
df['name'] = df['name'].str.split().str[:2].str.join(' ').str.strip()
df['brand'] = df['brand'].str.strip()
df['vehicle_transmission'] = df['vehicle_transmission'].str.strip()
df['fuel_type'] = df['fuel_type'].str.strip()

df.duplicated().sum() # yeh poorey dataframe mein dekhta hai ke wo row exactly repeat horahi hai tu count karta hai YANI SAARE COLUMNS KI VALUE KO DEKHTA HAI 

df = df.drop_duplicates()

df


# outliers mtlb jo extreme values hain column mein wo row hata dega or yeh sirf numeric columns par lagta hai
numeric_cols = ['year', 'mileage', 'prices', 'engine']

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Optional: Print stats
    print(f"{col}: Lower={lower_bound}, Upper={upper_bound}")

    # Remove outliers (row delete)
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    # OR: Cap the outliers instead of removing
    # df[col] = np.where(df[col] > upper_bound, upper_bound,
    #            np.where(df[col] < lower_bound, lower_bound, df[col]))

df



In [ ]:
df.to_csv('./output/clean.csv',index=False)

In [ ]:
df.groupby('brand')['prices'].mean().plot(kind='bar')
plt.xlabel("Brand")
plt.ylabel("Average Price")
plt.title("Average Car Price by Brand")
plt.show()


df.plot(kind='scatter', x='engine', y='prices')
plt.title("Engine Size vs Price")
plt.show()


df['fuel_type'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title("Fuel Type Distribution")
plt.show()


df.plot(kind='scatter', x='mileage', y='prices')
plt.title("Mileage vs Price")
plt.show()


df['year'].value_counts().sort_index().plot(kind='bar')
plt.title("Cars Count by Model Year")
plt.xlabel("Year")
plt.ylabel("Count")
plt.show()

In [ ]:
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-OU7EENT;"
    "Trusted_Connection=yes;"
)

with pyodbc.connect(conn_str,autocommit=True) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE DATABASE ETL')

conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-OU7EENT;"
    "Trusted_Connection=yes;"
    "Database=ETL;"
)


with pyodbc.connect(conn_str,autocommit=True) as conn:
    cursor = conn.cursor()
    cursor.execute(""" CREATE TABLE Pak_Wheels(
                            name varchar(50),
                            year int,
                            brand varchar(50),
                            mileage int,
                            prices int,
                            engine int,
                            vehicle_transmission varchar(50),
                            fuel_type varchar(50)
                   )

""")
    
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-OU7EENT;"
    "Trusted_Connection=yes;"
    "Database=ETL;"
)
    
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    for lab,val in df.iterrows():
        cursor.execute(""" INSERT INTO Pak_Wheels (name, year, brand, mileage, prices, engine, vehicle_transmission, fuel_type)
                       VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
        val['name'], val['year'], val['brand'], val['mileage'], val['prices'], val['engine'], val['vehicle_transmission'], val['fuel_type'])
    conn.commit()


with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Pak_Wheels')
    result = cursor.fetchall()
result